# 📓 Notebook 02: Data Exploration & Visualization
## Hệ thống Recommendation Phim

**Mục tiêu:**
1. Thống kê tổng quan dataset
2. Phân bố rating (histogram)
3. Top genres phổ biến (bar chart)
4. Top movies theo rating count và average rating
5. Rating heatmap
6. Phân bố số ratings theo user
7. Phân bố năm phát hành phim

---

## 1. Import Libraries

In [ ]:
# Import libraries
import os
import sys
import warnings
from collections import Counter

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive plots (optional)
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly not installed. Using matplotlib only.")

# MongoDB
from pymongo import MongoClient

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported successfully!")

## 2. Load Data from MongoDB

In [ ]:
# Configuration
MONGO_URI = "mongodb://localhost:27017"
DB_NAME = "movie_recommendation"

# Connect to MongoDB
print("🔗 Connecting to MongoDB...")
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
print(f"✅ Connected to: {DB_NAME}")

In [ ]:
# Load data into DataFrames
print("📥 Loading data from MongoDB...")

# Load movies
movies_cursor = db.movies.find({}, {'_id': 0})
movies_df = pd.DataFrame(list(movies_cursor))
print(f"  📽️ Movies: {len(movies_df):,}")

# Load users
users_cursor = db.users.find({}, {'_id': 0})
users_df = pd.DataFrame(list(users_cursor))
print(f"  👤 Users: {len(users_df):,}")

# Load ratings
ratings_cursor = db.ratings.find({}, {'_id': 0})
ratings_df = pd.DataFrame(list(ratings_cursor))
print(f"  ⭐ Ratings: {len(ratings_df):,}")

print("\n✅ Data loaded successfully!")

## 3. Overall Statistics

In [ ]:
# Overall Statistics
print("=" * 60)
print("📊 OVERALL DATASET STATISTICS")
print("=" * 60)

total_movies = len(movies_df)
total_users = len(users_df)
total_ratings = len(ratings_df)
avg_rating = ratings_df['rating'].mean()
ratings_per_user = total_ratings / total_users if total_users > 0 else 0
ratings_per_movie = total_ratings / total_movies if total_movies > 0 else 0
sparsity = 1 - (total_ratings / (total_movies * total_users)) if (total_movies * total_users) > 0 else 0

print(f"""
📽️ Total Movies:           {total_movies:,}
👤 Total Users:            {total_users:,}
⭐ Total Ratings:          {total_ratings:,}

📈 Average Rating:         {avg_rating:.2f}
📊 Ratings per User:       {ratings_per_user:.1f}
📊 Ratings per Movie:      {ratings_per_movie:.1f}
🔲 Matrix Sparsity:        {sparsity:.2%}
""")

## 4. Rating Distribution (Histogram)

In [ ]:
# Rating Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
rating_counts = ratings_df['rating'].value_counts().sort_index()
axes[0].bar(rating_counts.index, rating_counts.values, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rating', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('📊 Rating Distribution', fontsize=14, fontweight='bold')
axes[0].set_xticks([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

# Add value labels on bars
for i, (rating, count) in enumerate(rating_counts.items()):
    axes[0].text(rating, count + max(rating_counts.values)*0.01, f'{count:,}', 
                 ha='center', va='bottom', fontsize=8)

# Box plot
axes[1].boxplot(ratings_df['rating'], vert=True)
axes[1].set_ylabel('Rating', fontsize=12)
axes[1].set_title('📦 Rating Box Plot', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Statistics
print(f"\n📈 Rating Statistics:")
print(ratings_df['rating'].describe())

## 5. Top Genres (Bar Chart)

In [ ]:
# Count genres
all_genres = []
for genres in movies_df['genres']:
    if isinstance(genres, list):
        all_genres.extend(genres)

genre_counts = Counter(all_genres)
top_genres = pd.DataFrame(genre_counts.most_common(15), columns=['Genre', 'Count'])

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(top_genres['Genre'], top_genres['Count'], color=sns.color_palette("viridis", len(top_genres)))
ax.set_xlabel('Number of Movies', fontsize=12)
ax.set_ylabel('Genre', fontsize=12)
ax.set_title('🎬 Top 15 Genres by Number of Movies', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for bar, count in zip(bars, top_genres['Count']):
    ax.text(bar.get_width() + max(top_genres['Count'])*0.01, bar.get_y() + bar.get_height()/2, 
            f'{count:,}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n📊 Total unique genres: {len(genre_counts)}")

## 6. Top Movies by Rating Count

In [ ]:
# Top movies by rating count
top_by_count = movies_df.nlargest(20, 'ratingCount')[['title', 'year', 'ratingCount', 'avgRating']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(top_by_count)), top_by_count['ratingCount'], 
               color=sns.color_palette("rocket", len(top_by_count)))
ax.set_yticks(range(len(top_by_count)))
ax.set_yticklabels([f"{t[:40]}..." if len(t) > 40 else t for t in top_by_count['title']], fontsize=9)
ax.set_xlabel('Number of Ratings', fontsize=12)
ax.set_title('🏆 Top 20 Movies by Rating Count', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (count, avg) in enumerate(zip(top_by_count['ratingCount'], top_by_count['avgRating'])):
    ax.text(count + max(top_by_count['ratingCount'])*0.01, i, 
            f'{count:,} (⭐{avg:.1f})', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 7. Top Movies by Average Rating (min 100 ratings)

In [ ]:
# Top movies by average rating (minimum 100 ratings to qualify)
min_ratings = 100
qualified_movies = movies_df[movies_df['ratingCount'] >= min_ratings]
top_by_rating = qualified_movies.nlargest(20, 'avgRating')[['title', 'year', 'avgRating', 'ratingCount']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(top_by_rating)), top_by_rating['avgRating'], 
               color=sns.color_palette("YlOrRd", len(top_by_rating)))
ax.set_yticks(range(len(top_by_rating)))
ax.set_yticklabels([f"{t[:40]}..." if len(t) > 40 else t for t in top_by_rating['title']], fontsize=9)
ax.set_xlabel('Average Rating', fontsize=12)
ax.set_xlim(0, 5.5)
ax.set_title(f'⭐ Top 20 Movies by Average Rating (min {min_ratings} ratings)', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (avg, count) in enumerate(zip(top_by_rating['avgRating'], top_by_rating['ratingCount'])):
    ax.text(avg + 0.05, i, f'{avg:.2f} ({count:,} ratings)', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 8. Rating Heatmap (Sample Users x Movies)

In [ ]:
# Create rating heatmap for sample users and movies
n_users_sample = 30
n_movies_sample = 30

# Get most active users
user_rating_counts = ratings_df.groupby('userId').size().sort_values(ascending=False)
top_users = user_rating_counts.head(n_users_sample).index.tolist()

# Get most rated movies
movie_rating_counts = ratings_df.groupby('movieId').size().sort_values(ascending=False)
top_movies = movie_rating_counts.head(n_movies_sample).index.tolist()

# Filter ratings
sample_ratings = ratings_df[
    (ratings_df['userId'].isin(top_users)) & 
    (ratings_df['movieId'].isin(top_movies))
]

# Create pivot table
rating_matrix = sample_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Plot heatmap
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(rating_matrix, cmap='YlOrRd', center=3, 
            cbar_kws={'label': 'Rating'}, ax=ax,
            xticklabels=False, yticklabels=True)
ax.set_xlabel('Movies', fontsize=12)
ax.set_ylabel('Users', fontsize=12)
ax.set_title(f'🔥 Rating Heatmap (Top {n_users_sample} Users × Top {n_movies_sample} Movies)', 
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Calculate fill rate
fill_rate = sample_ratings.shape[0] / (len(top_users) * len(top_movies)) * 100
print(f"\n📊 Heatmap fill rate: {fill_rate:.1f}%")

## 9. User Activity Distribution

In [ ]:
# User activity distribution
user_activity = ratings_df.groupby('userId').size()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(user_activity, bins=50, color='teal', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Ratings', fontsize=12)
axes[0].set_ylabel('Number of Users', fontsize=12)
axes[0].set_title('👤 User Activity Distribution', fontsize=14, fontweight='bold')
axes[0].set_yscale('log')

# Box plot
axes[1].boxplot(user_activity, vert=True)
axes[1].set_ylabel('Number of Ratings', fontsize=12)
axes[1].set_title('📦 User Activity Box Plot', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 User Activity Statistics:")
print(user_activity.describe())

## 10. Movie Release Year Distribution

In [ ]:
# Movie release year distribution
year_counts = movies_df['year'].dropna().astype(int).value_counts().sort_index()

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(year_counts.index, year_counts.values, color='coral', edgecolor='black', alpha=0.7, width=1)
ax.set_xlabel('Release Year', fontsize=12)
ax.set_ylabel('Number of Movies', fontsize=12)
ax.set_title('📅 Movies by Release Year', fontsize=14, fontweight='bold')

# Add decade markers
decades = range(1900, 2030, 10)
ax.set_xticks(decades)
ax.set_xticklabels([str(d) for d in decades], rotation=45)

plt.tight_layout()
plt.show()

# Statistics
print(f"\n📊 Year Statistics:")
print(f"  Earliest: {movies_df['year'].min():.0f}")
print(f"  Latest:   {movies_df['year'].max():.0f}")
print(f"  Most movies in: {year_counts.idxmax()} ({year_counts.max():,} movies)")

## 11. Average Rating by Genre

In [ ]:
# Average rating by genre
genre_ratings = {}

for _, movie in movies_df.iterrows():
    if movie['ratingCount'] > 0 and isinstance(movie['genres'], list):
        for genre in movie['genres']:
            if genre not in genre_ratings:
                genre_ratings[genre] = []
            genre_ratings[genre].append(movie['avgRating'])

# Calculate average
genre_avg = {g: np.mean(r) for g, r in genre_ratings.items()}
genre_df = pd.DataFrame([
    {'Genre': g, 'AvgRating': r, 'MovieCount': len(genre_ratings[g])} 
    for g, r in genre_avg.items()
]).sort_values('AvgRating', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
colors = sns.color_palette("RdYlGn", len(genre_df))
bars = ax.barh(genre_df['Genre'], genre_df['AvgRating'], color=colors)
ax.set_xlabel('Average Rating', fontsize=12)
ax.set_xlim(0, 5)
ax.set_title('⭐ Average Rating by Genre', fontsize=14, fontweight='bold')
ax.axvline(x=avg_rating, color='red', linestyle='--', label=f'Overall Avg: {avg_rating:.2f}')
ax.legend()

# Add value labels
for bar, (rating, count) in zip(bars, zip(genre_df['AvgRating'], genre_df['MovieCount'])):
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2, 
            f'{rating:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 12. Rating Trends Over Time

In [ ]:
# Rating trends over time (by movie release year)
year_rating = movies_df[movies_df['ratingCount'] >= 10].groupby('year').agg(
    avgRating=('avgRating', 'mean'),
    movieCount=('movieId', 'count'),
    totalRatings=('ratingCount', 'sum')
).reset_index()

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Average rating by year
axes[0].plot(year_rating['year'], year_rating['avgRating'], 'o-', color='steelblue', markersize=3)
axes[0].set_xlabel('Release Year', fontsize=12)
axes[0].set_ylabel('Average Rating', fontsize=12)
axes[0].set_title('📈 Average Rating Trend by Release Year', fontsize=14, fontweight='bold')
axes[0].axhline(y=avg_rating, color='red', linestyle='--', alpha=0.7, label=f'Overall Avg: {avg_rating:.2f}')
axes[0].legend()
axes[0].set_ylim(2.5, 4.5)

# Number of ratings by year
axes[1].bar(year_rating['year'], year_rating['totalRatings'], color='coral', alpha=0.7, width=1)
axes[1].set_xlabel('Release Year', fontsize=12)
axes[1].set_ylabel('Total Ratings', fontsize=12)
axes[1].set_title('📊 Total Ratings by Release Year', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## ✅ Summary

In [ ]:
# Summary
print("=" * 60)
print("🎉 DATA EXPLORATION COMPLETED!")
print("=" * 60)

print(f"""
📊 Key Insights:

1. Dataset Overview:
   - {total_movies:,} movies, {total_users:,} users, {total_ratings:,} ratings
   - Average rating: {avg_rating:.2f}
   - Matrix sparsity: {sparsity:.2%}

2. Rating Distribution:
   - Most ratings are between 3.0 and 4.0
   - Mode rating: {ratings_df['rating'].mode()[0]}

3. Top Genres:
   - {top_genres.iloc[0]['Genre']} ({top_genres.iloc[0]['Count']:,} movies)
   - {top_genres.iloc[1]['Genre']} ({top_genres.iloc[1]['Count']:,} movies)
   - {top_genres.iloc[2]['Genre']} ({top_genres.iloc[2]['Count']:,} movies)

4. User Activity:
   - Average ratings per user: {user_activity.mean():.1f}
   - Most active user: {user_activity.max():,} ratings

5. Movie Years:
   - Range: {movies_df['year'].min():.0f} - {movies_df['year'].max():.0f}
   - Peak year: {year_counts.idxmax()} ({year_counts.max():,} movies)

✅ Next Steps:
   - Run Notebook 03: Model Training
   - Run Notebook 04: Model Evaluation
""")

# Close MongoDB connection
client.close()
print("🔌 MongoDB connection closed.")